<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/Analyze_Company_Financials_With_Polygon_io_Actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Polygon.IO
* Polygon.IO is a super duper data platform [Polygon.IO](https://polygon.io/)

* Go to Pricing and subscribe to Free Account
    - 4 types of subscriptions for each Options, Stocks, Indices, Forex
    - Free tier is good enough for a lot. 5 API calls max per minute
* For docs go to docs / client library Python. See [Polygon.IO Docs](https://polygon.io/docs/stocks/getting-started)

### Prerequisite Video

### Get Historical Stock Data From Polygon.IO API (click to view)


[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/6TeHBXbvkPI/0.jpg)](https://www.youtube.com/watch?v=6TeHBXbvkPI)

### Polygon.IO Playlist

[![IMAGE ALT TEXT HERE](https://asset.brandfetch.io/idfGjoDDfo/idGLhETxky.png)](https://www.youtube.com/playlist?list=PLCZZtBmmgxn-QlbbMb-g-89GjHiml_AtS
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 650.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


#### Install Client Library


In [ ]:
!pip install polygon-api-client

### API Key
* Get in dashboard https://polygon.io/dashboard
* Copy and paste here to MY_API_KEY. Note I did the "proper" thing by setting a secret, but this isn't necessary and you can just set MY_API_KEY to your API Key string

In [2]:
from google.colab import userdata
MY_API_KEY = userdata.get('POLYGON_API_KEY')

#### Connect to client

In [6]:
from polygon import RESTClient
client = RESTClient(MY_API_KEY)

### Company Financials "Fundamental Data"
#### Use *client.vx.list_stock_financials* to get OHLCV data for all stocks!
* Pagenated so we should set limit to MAX_LIMIT=100 since free tier has 5 API calls per minute


In [7]:
help(client.vx.list_stock_financials)

Help on method list_stock_financials in module polygon.rest.vX:

list_stock_financials(ticker: Optional[str] = None, cik: Optional[str] = None, company_name: Optional[str] = None, company_name_search: Optional[str] = None, sic: Optional[str] = None, filing_date: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, filing_date_lt: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, filing_date_lte: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, filing_date_gt: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, filing_date_gte: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, period_of_report_date: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, period_of_report_date_lt: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, period_of_report_date_lte: Union[str, int, datetime.datetime, datetime.date, NoneType] = None, period_of_report_date_gt: Union[str, int, datetime.

In [8]:
import time
financials = []
MAX_LIMIT = 100
for f in client.vx.list_stock_financials(ticker='TSLA',filing_date_gte='2019-01-01',limit=MAX_LIMIT):
  financials.append(f)
  if (len(financials) % (MAX_LIMIT*5) == 0):
    time.sleep(60)
    print(f'sleeping for 60 seconds')



In [10]:
len(financials)

22

##### Turn into a dataframe for readability

In [11]:
import pandas as pd
df = pd.DataFrame(financials)
df.head()

,cik,company_name,end_date,filing_date,financials,fiscal_period,fiscal_year,source_filing_file_url,source_filing_url,start_date
0,0001318605,"Tesla, Inc.",2024-03-31,2024-04-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q1,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-01-01
1,0001318605,"Tesla, Inc.",2023-12-31,2024-01-29,{'balance_sheet': {'redeemable_noncontrolling_...,FY,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01
2,0001318605,"Tesla, Inc.",2023-09-30,2023-10-23,"{'balance_sheet': {'assets': {'formula': None,...",Q3,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-07-01
3,0001318605,"Tesla, Inc.",2023-06-30,2023-07-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q2,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-04-01
4,0001318605,"Tesla, Inc.",2023-03-31,2023-04-24,{'balance_sheet': {'current_assets': {'formula...,Q1,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01


#### What does the "financials" column have?

In [16]:
financials_keys=list(df.financials.iloc[0].keys())
financials_keys

['balance_sheet',
 'cash_flow_statement',
 'comprehensive_income',
 'income_statement']

### Let's parse them out
* Note this isn't perfect and I am oversimplifying things because there can be different dicts with different keys...

In [17]:
for key in financials_keys:
  df[key] = df.financials.apply(lambda x: x[key])

In [18]:
df

,cik,company_name,end_date,filing_date,financials,fiscal_period,fiscal_year,source_filing_file_url,source_filing_url,start_date,balance_sheet,cash_flow_statement,comprehensive_income,income_statement
0,0001318605,"Tesla, Inc.",2024-03-31,2024-04-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q1,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-01-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
1,0001318605,"Tesla, Inc.",2023-12-31,2024-01-29,{'balance_sheet': {'redeemable_noncontrolling_...,FY,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
2,0001318605,"Tesla, Inc.",2023-09-30,2023-10-23,"{'balance_sheet': {'assets': {'formula': None,...",Q3,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-07-01,"{'assets': {'formula': None, 'label': 'Assets'...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
3,0001318605,"Tesla, Inc.",2023-06-30,2023-07-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q2,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-04-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
4,0001318605,"Tesla, Inc.",2023-03-31,2023-04-24,{'balance_sheet': {'current_assets': {'formula...,Q1,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
5,0001318605,"Tesla, Inc.",2022-12-31,2023-01-31,{'balance_sheet': {'liabilities': {'formula': ...,FY,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-01-01,"{'liabilities': {'formula': None, 'label': 'Li...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
6,0001318605,"Tesla, Inc.",2022-09-30,2022-10-24,{'balance_sheet': {'current_assets': {'formula...,Q3,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-07-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
7,0001318605,"Tesla, Inc.",2022-06-30,2022-07-25,{'balance_sheet': {'equity_attributable_to_non...,Q2,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-04-01,{'equity_attributable_to_noncontrolling_intere...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,..."
8,0001318605,"Tesla, Inc.",2022-03-31,2022-04-25,{'balance_sheet': {'noncurrent_liabilities': {...,Q1,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-01-01,"{'noncurrent_liabilities': {'formula': None, '...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': 

##### Example income_statement entry

In [20]:
df.income_statement.iloc[0]

{'basic_earnings_per_share': {'formula': None,
  'label': 'Basic Earnings Per Share',
  'order': 4200,
  'unit': 'USD / shares',
  'value': 0.37,
  'xpath': None},
 'cost_of_revenue': {'formula': None,
  'label': 'Cost Of Revenue',
  'order': 300,
  'unit': 'USD',
  'value': 17605000000.0,
  'xpath': None},
 'gross_profit': {'formula': None,
  'label': 'Gross Profit',
  'order': 800,
  'unit': 'USD',
  'value': 3696000000.0,
  'xpath': None},
 'operating_expenses': {'formula': None,
  'label': 'Operating Expenses',
  'order': 1000,
  'unit': 'USD',
  'value': 2525000000.0,
  'xpath': None},
 'revenues': {'formula': None,
  'label': 'Revenues',
  'order': 100,
  'unit': 'USD',
  'value': 21301000000.0,
  'xpath': None}}

### Let's see what keys each of these new columns has

In [24]:
from IPython.display import display, Markdown
for key in financials_keys:
  display(Markdown(f'### {key}'))
  print("\n".join(df[key].loc[0].keys()))

### balance_sheet

redeemable_noncontrolling_interest
equity
assets
liabilities
equity_attributable_to_noncontrolling_interest
long_term_debt
noncurrent_assets
accounts_payable
inventory
current_assets
current_liabilities
equity_attributable_to_parent
temporary_equity
other_noncurrent_assets
other_noncurrent_liabilities
other_current_liabilities
liabilities_and_equity
noncurrent_liabilities
wages
intangible_assets
fixed_assets
other_current_assets


### cash_flow_statement

exchange_gains_losses
net_cash_flow
net_cash_flow_from_financing_activities


### comprehensive_income

comprehensive_income_loss
comprehensive_income_loss_attributable_to_parent
other_comprehensive_income_loss


### income_statement

basic_earnings_per_share
cost_of_revenue
gross_profit
operating_expenses
revenues


### Let's parse out things from the income statement

#### Note we need the 'value' field to get the values

In [26]:
for key in df['income_statement'].iloc[0].keys():
  df[key] = df.income_statement.apply(lambda x: x[key]['value'])
df

,cik,company_name,end_date,filing_date,financials,fiscal_period,fiscal_year,source_filing_file_url,source_filing_url,start_date,balance_sheet,cash_flow_statement,comprehensive_income,income_statement,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
0,0001318605,"Tesla, Inc.",2024-03-31,2024-04-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q1,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-01-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.37,1.760500e+10,3.696000e+09,2.525000e+09,2.130100e+10
1,0001318605,"Tesla, Inc.",2023-12-31,2024-01-29,{'balance_sheet': {'redeemable_noncontrolling_...,FY,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",4.73,7.911300e+10,1.766000e+10,8.769000e+09,9.677300e+10
2,0001318605,"Tesla, Inc.",2023-09-30,2023-10-23,"{'balance_sheet': {'assets': {'formula': None,...",Q3,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-07-01,"{'assets': {'formula': None, 'label': 'Assets'...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.58,1.917200e+10,4.178000e+09,2.414000e+09,2.335000e+10
3,0001318605,"Tesla, Inc.",2023-06-30,2023-07-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q2,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-04-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.85,2.039400e+10,4.533000e+09,2.134000e+09,2.492700e+10
4,0001318605,"Tesla, Inc.",2023-03-31,2023-04-24,{'balance_sheet': {'current_assets': {'formula...,Q1,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.80,1.881800e+10,4.511000e+09,1.847000e+09,2.332900e+10
5,0001318605,"Tesla, Inc.",2022-12-31,2023-01-31,{'balance_sheet': {'liabilities': {'formula': ...,FY,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-01-01,"{'liabilities': {'formula': None, 'label': 'Li...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",4.02,6.060900e+10,2.085300e+10,7.197000e+09,8.146200e+10
6,0001318605,"Tesla, Inc.",2022-09-30,2022-10-24,{'balance_sheet': {'current_assets': {'formula...,Q3,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-07-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",1.05,1.607200e+10,5.382000e+09,1.694000e+09,2.145400e+10
7,0001318605,"Tesla, Inc.",2022-06-30,2022-07-25,{'balance_sheet': {'equity_attributable_to_non...,Q2,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-04-01,{'equity_attributable_to_noncontrolling_intere...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'for

### Plot Quarterly EPS,Revenue, and Gross Profit Over Time

In [27]:
quarterly_financials = df[df.fiscal_period.str.contains('Q')]
quarterly_financials

,cik,company_name,end_date,filing_date,financials,fiscal_period,fiscal_year,source_filing_file_url,source_filing_url,start_date,balance_sheet,cash_flow_statement,comprehensive_income,income_statement,basic_earnings_per_share,cost_of_revenue,gross_profit,operating_expenses,revenues
0,0001318605,"Tesla, Inc.",2024-03-31,2024-04-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q1,2024,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2024-01-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.37,1.760500e+10,3.696000e+09,2.525000e+09,2.130100e+10
2,0001318605,"Tesla, Inc.",2023-09-30,2023-10-23,"{'balance_sheet': {'assets': {'formula': None,...",Q3,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-07-01,"{'assets': {'formula': None, 'label': 'Assets'...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.58,1.917200e+10,4.178000e+09,2.414000e+09,2.335000e+10
3,0001318605,"Tesla, Inc.",2023-06-30,2023-07-24,{'balance_sheet': {'redeemable_noncontrolling_...,Q2,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-04-01,{'redeemable_noncontrolling_interest': {'formu...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.85,2.039400e+10,4.533000e+09,2.134000e+09,2.492700e+10
4,0001318605,"Tesla, Inc.",2023-03-31,2023-04-24,{'balance_sheet': {'current_assets': {'formula...,Q1,2023,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2023-01-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",0.80,1.881800e+10,4.511000e+09,1.847000e+09,2.332900e+10
6,0001318605,"Tesla, Inc.",2022-09-30,2022-10-24,{'balance_sheet': {'current_assets': {'formula...,Q3,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-07-01,"{'current_assets': {'formula': None, 'label': ...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",1.05,1.607200e+10,5.382000e+09,1.694000e+09,2.145400e+10
7,0001318605,"Tesla, Inc.",2022-06-30,2022-07-25,{'balance_sheet': {'equity_attributable_to_non...,Q2,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-04-01,{'equity_attributable_to_noncontrolling_intere...,"{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",2.18,1.270000e+10,4.234000e+09,1.770000e+09,1.693400e+10
8,0001318605,"Tesla, Inc.",2022-03-31,2022-04-25,{'balance_sheet': {'noncurrent_liabilities': {...,Q1,2022,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2022-01-01,"{'noncurrent_liabilities': {'formula': None, '...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'formula': None...,"{'basic_earnings_per_share': {'formula': None,...",3.20,1.329600e+10,5.460000e+09,1.857000e+09,1.875600e+10
10,0001318605,"Tesla, Inc.",2021-09-30,2021-10-25,{'balance_sheet': {'accounts_payable': {'formu...,Q3,2021,http://api.polygon.io/v1/reference/sec/filings...,https://api.polygon.io/v1/reference/sec/filing...,2021-07-01,"{'accounts_payable': {'formula': None, 'label'...","{'exchange_gains_losses': None, 'net_cash_flow...",{'comprehensive_income_loss': {'

### Convert to tall format for plotly express using Pandas Melt

##### Let's use pandas' melt function to convert to a tall format so we can use plotly express
* Instead of columns gross_profit, revenue, basic_earnings_per_share as seperate columns we want one column with all these values and a variable called measurement_type which will take values in ['gross_profit','revenues','basic_earnings_per_share']

In [66]:
quarterly_financials_tall=quarterly_financials.melt(id_vars=['start_date','end_date'],value_vars=['gross_profit','revenues','basic_earnings_per_share','operating_expenses'],var_name='Financial Metric',value_name='Dollars')

### Plot these 3 measurements with plotly express

In [67]:
import plotly.express as px

fig = px.line(quarterly_financials_tall,x='start_date',y='Dollars',facet_col='Financial Metric',facet_col_wrap=1,height=800)

fig.update_yaxes(matches=None)

fig.show()

### Let's Add Nvidia so we can compare
* First let's wrap all the parsing above in a function

In [68]:
def get_quarterly_financials(symbol):
    financials = []
    MAX_LIMIT = 100
    for f in client.vx.list_stock_financials(symbol,filing_date_gte='2019-01-01',limit=MAX_LIMIT):
        financials.append(f)
        if len(financials) % (5*MAX_LIMIT) == 0:
            time.sleep(60)
    df_financials = pd.DataFrame(financials)
    financials_keys=list(df_financials.financials.iloc[0].keys())
    for key in financials_keys:
        df_financials[key]=df_financials.financials.apply(lambda l:l[key])
    for key in df_financials['income_statement'].iloc[0].keys():
        df_financials[key]=df_financials['income_statement'].apply(lambda l:l[key]['value'])
    quarterly_financials=df_financials[df_financials.fiscal_period.str.contains('Q')]
    quarterly_financials_tall=quarterly_financials.melt(id_vars=['start_date','end_date'],
                                                    value_vars=['gross_profit','revenues','basic_earnings_per_share','operating_expenses'],
                                                    var_name='Financial Metric',value_name='Dollars')
    return quarterly_financials_tall

In [69]:
quarterly_financials_NVDA = get_quarterly_financials('NVDA')

In [70]:
quarterly_financials_NVDA

,start_date,end_date,Financial Metric,Dollars
0,2024-01-29,2024-04-28,gross_profit,2.040600e+10
1,2023-07-31,2023-10-29,gross_profit,1.340000e+10
2,2023-05-01,2023-07-30,gross_profit,9.462000e+09
3,2023-01-30,2023-04-30,gross_profit,4.648000e+09
4,2022-08-01,2022-10-30,gross_profit,3.177000e+09
...,...,...,...,...
59,2020-04-27,2020-07-26,operating_expenses,1.624000e+09
60,2020-01-27,2020-04-26,operating_expenses,1.028000e+09
61,2019-07-29,2019-10-27,operating_expenses,9.890000e+08
62,2019-04-29,2019-07-28,operating_expenses,9.700000e+08


### Combine with TSLA for comparison

In [75]:
quarterly_financials_NVDA_TSLA = pd.concat([quarterly_financials_NVDA.assign(symbol='NVDA'),quarterly_financials_tall.assign(symbol='TSLA')])

In [76]:
quarterly_financials_NVDA_TSLA

,start_date,end_date,Financial Metric,Dollars,symbol
0,2024-01-29,2024-04-28,gross_profit,2.040600e+10,NVDA
1,2023-07-31,2023-10-29,gross_profit,1.340000e+10,NVDA
2,2023-05-01,2023-07-30,gross_profit,9.462000e+09,NVDA
3,2023-01-30,2023-04-30,gross_profit,4.648000e+09,NVDA
4,2022-08-01,2022-10-30,gross_profit,3.177000e+09,NVDA
...,...,...,...,...,...
59,2020-04-01,2020-06-30,operating_expenses,9.400000e+08,TSLA
60,2020-01-01,2020-03-31,operating_expenses,9.510000e+08,TSLA
61,2019-07-01,2019-09-30,operating_expenses,9.300000e+08,TSLA
62,2019-04-01,2019-06-30,operating_expenses,1.088504e+09,TSLA


In [81]:
import plotly.express as px

fig = px.line(quarterly_financials_NVDA_TSLA,x='start_date',y='Dollars',facet_col='Financial Metric',color='symbol',facet_col_wrap=2,height=800,width=800)

fig.update_yaxes(matches=None)

# Custom titles for each facet
custom_titles = {
    'Financial Metric=revenues': 'Revenue',
    'Financial Metric=gross_profit': 'Gross Profit',
    'Financial Metric=basic_earnings_per_share': 'Earnings Per Share',
    'Financial Metric=operating_expenses': 'Operating Expenses'
}

fig.for_each_annotation(lambda a: a.update(text=custom_titles.get(a.text, a.text)))

# Ensure all x-axes show tick labels
for i in range(len(fig.layout.annotations)):
    fig.layout[f'xaxis{i+1}'].showticklabels = True



fig.show()